Check the pointwise convergence of the reduction constraints. First, you need to run a simulation at least in three different resolutions (and double resolution every time, otherwise you need to modify the script). The script assumes that you saved data only at the common timesteps.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/classical_runs/"
par = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.0_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp0"
your_dir = dir*par

Uncomment below if you want to save your plots.

In [ ]:
#D = 3
#out_dir = "./convergence_plots/"*par*"/resol_D$(D)$(D+1)$(D+2)"
#if ispath(out_dir)==false
#    mkpath(out_dir)
#end

In [ ]:
D=3 # give D of the smallest (coarse) resolution
Nr = 128*2^D + 3 # the overal course graining
n = 0

# load the r grid
r_c = h5read(your_dir*"/data_$((Nr-3)*2^n+3)/r.h5","r")
drc = r_c[2] - r_c[1]
r_m = h5read(your_dir*"/data_$((Nr-3)*2^(n+1)+3)/r.h5","r")
r_f = h5read(your_dir*"/data_$((Nr-3)*2^(n+2)+3)/r.h5","r")

# list all available iterations (and corresponding files)
(its_c, all_filenames_c) = list_h5_files(your_dir*"/data_$((Nr-3)*2^n+3)", prefix="data_")
(its_m, all_filenames_m) = list_h5_files(your_dir*"/data_$((Nr-3)*2^(n+1)+3)", prefix="data_")
(its_f, all_filenames_f) = list_h5_files(your_dir*"/data_$((Nr-3)*2^(n+2)+3)", prefix="data_")

# lists used in checking if the projection on the coarse grid is done correctly
@assert r_c[3:end] ≈ r_m[3:2:end]
@assert r_c[3:end] ≈ r_f[3:4:end]

In [ ]:
# get index of r_c in coarse resol, for which  r[index] is the outermost causally connected point to rmax
it_c      = its_c[end]
it_str_c  = lpad(it_c, 4, "0")
tc_end    =  h5readattr(your_dir*"/data_$((Nr-3)*2^n + 3)/data_$(it_str_c).h5", "/")["time"]
r_c[end] - tc_end
ri = 1
i = 1
while r_c[i] != r_c[end] - tc_end
    i += 1
end
ri_max_causal = i-1
drc = r_c[4] - r_c[3]
r_c[ri_max_causal] == r_c[end] - tc_end - drc
ri_max_causal

In [ ]:
#choose the commmon timestep to do the comparison.
#remember that its_c =its_m[1:2:end] = its_c[1:4:end]
#it_c is the timestep of the coarse resolution    
i = 15
it_c = its_c[i]#[i] for when they are saved on common timesteps
it_m = its_m[i]
it_f = its_f[i]
it_str_c  = lpad(it_c, 4, "0")
it_str_m  = lpad(it_m, 4, "0")
it_str_f  = lpad(it_f, 4, "0")

# v_classic = [Φ, Π, Ψ, A, B, DB, Utld, K, KB, λ, α, Dα, Θ, Zr, f, g, U, V]^T
v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
# load data
vc = h5read(your_dir*"/data_$((Nr-3)*2^n + 3)/data_$(it_str_c).h5","v")
vm = h5read(your_dir*"/data_$((Nr-3)*2^(n+1) + 3)/data_$(it_str_m).h5","v")
vf = h5read(your_dir*"/data_$((Nr-3)*2^(n+2) + 3)/data_$(it_str_f).h5","v")

tc = h5readattr(your_dir*"/data_$((Nr-3)*2^n + 3)/data_$(it_str_c).h5", "/")["time"]
tm = h5readattr(your_dir*"/data_$((Nr-3)*2^(n+1) + 3)/data_$(it_str_m).h5", "/")["time"]
tf = h5readattr(your_dir*"/data_$((Nr-3)*2^(n+2) + 3)/data_$(it_str_f).h5", "/")["time"]

Φc    = vc[:,1]
Πc    = vc[:,2]
Ψc    = vc[:,3]
Ac    = vc[:,4]
Bc    = vc[:,5]
DBc   = vc[:,6]
Utldc = vc[:,7]
Kc    = vc[:,8]
KBc   = vc[:,9]
λc    = vc[:,10]
αc    = vc[:,11]
Dαc   = vc[:,12]

Φm    = vm[:,1]
Πm    = vm[:,2]
Ψm    = vm[:,3]
Am    = vm[:,4]
Bm    = vm[:,5]
DBm   = vm[:,6]
Utldm = vm[:,7]
Km    = vm[:,8]
KBm   = vm[:,9]
λm    = vm[:,10]
αm    = vm[:,11]
Dαm   = vm[:,12]

Φf    = vf[:,1]
Πf    = vf[:,2]
Ψf    = vf[:,3]
Af    = vf[:,4]
Bf    = vf[:,5]
DBf   = vf[:,6]
Utldf = vf[:,7]
Kf    = vf[:,8]
KBf   = vf[:,9]
λf    = vf[:,10]
αf    = vf[:,11]
Dαf   = vf[:,12]

# make sure we're comparing the same timestep
@assert tc ≈ tm
@assert tc ≈ tf

In [ ]:
# function to reconstruct the reduction variable from saved data
function reconstruct_Df(r::Array, f::Array)
    dr = r[4] - r[3]
    Df = Dr_FD2(f,dr)./f
    return Df
end

Reconstruct the reduction variables:

In [ ]:
rec_DAc = reconstruct_Df(r_c, Ac);
rec_DAm = reconstruct_Df(r_m, Am);
rec_DAf = reconstruct_Df(r_f, Af);

rec_DBc = reconstruct_Df(r_c, Bc);
rec_DBm = reconstruct_Df(r_m, Bm);
rec_DBf = reconstruct_Df(r_f, Bf);

rec_Utldc = @. rec_DAc - 2.0*rec_DBc - 4.0*Bc*λc/Ac
rec_Utldm = @. rec_DAm - 2.0*rec_DBm - 4.0*Bm*λm/Am
rec_Utldf = @. rec_DAf - 2.0*rec_DBf - 4.0*Bf*λf/Af

rec_Dαc = reconstruct_Df(r_c, αc);
rec_Dαm = reconstruct_Df(r_m, αm);
rec_Dαf = reconstruct_Df(r_f, αf);

Plot the reconstructed and saved reduction variable, together with the location of the apparent horizon (if it is negative, then there is none).

In [ ]:
AH = find_AH(r_c, Bc, Ac, KBc)
println("r_AH = ", AH)

plot(r_c, DBc, title = "t=$(tc)",  label="DB_c(r)", linewidth = 2)
plot!(r_c, rec_DBc, title = "t=$(tc)",  label="rec_DBc(r)", linewidth = 2,
ylim=(-0.5,0.5), xlim=(0,5))

plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)


Plot the difference of the reconstructed and saved reduction variable, in the three different resolutions. The rescaling factor 4 and 4^2=16 are related to the construction of these convergence tests (doubling resolution) and the 2nd order accurate finite difference operators. If there is good overlap ot the rescaled errors, then we have good pointwise 2nd order convergence.  

In [ ]:
AH = find_AH(r_f, Bf, Af, KBf)
println("r_AH = ", AH)

plot(r_c, DBc-rec_DBc, title = "t=$(tc)",  label="diff_DB_c(r)", linewidth = 2)#, xlim=(0,5))
plot!(r_m, 4*(DBm-rec_DBm), label="4*diff_DBm(r)", linewidth = 2)
plot!(r_f, 16*(DBf-rec_DBf), label="16*diff_DBm(r)", linewidth = 2)

plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)


Repeat for more variables (their diffs are not rescaled, but can be done easily, as above).

In [ ]:
r = r_c
ri_max = ri_max_causal

AH = find_AH(r_c, Bc, Ac, KBc)
println("r_AH = ", AH)

plot(r_c[3:ri_max], Dαc[3:ri_max], title = "t=$(tc)",  label="Dαc(r)", linewidth = 2)
plot!(r_c[3:ri_max], rec_Dαc[3:ri_max], title = "t=$(tc)",  label="rec_Dαc(r)", linewidth = 2,
    #ylim=(-0.5,0.5),
    #xlim=(0,5)
)
plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)


In [ ]:
AH = find_AH(r_f, Bf, Af, KBf)
println("r_AH = ", AH)

plot(r_c, Dαc-rec_Dαc, title = "t=$(tc)",  label="diff_Dαc(r)", linewidth = 2, xlim=(-1,15))
plot!(r_m, Dαm-rec_Dαm, label="diff_Dαm(r)", linewidth = 2)
plot!(r_f, Dαf-rec_Dαf, label="diff_Dαm(r)", linewidth = 2)

#plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)


In [ ]:
AH = find_AH(r_c, Bc, Ac, KBc)
println("r_AH = ", AH)

plot(r_c, Utldc, title = "t=$(tc)",  label="Utld_c(r)", linewidth = 2)
plot!(r_c, rec_Utldc, title = "t=$(tc)",  label="rec_Utldc(r)", linewidth = 2
#ylim=(-0.5,0.5), xlim=(0,5)
)

plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)

In [ ]:
AH = find_AH(r_f, Bf, Af, KBf)
println("r_AH = ", AH)

plot(r_c, Utldc-rec_Utldc, title = "t=$(tc)",  label="diff_Utldc(r)", linewidth = 2, xlim=(-1,15))
plot!(r_m, Utldm-rec_Utldm, label="diff_Utldm(r)", linewidth = 2)
plot!(r_f, Utldf-rec_Utldf, label="diff_Utldm(r)", linewidth = 2)

#plot!([AH], seriestype="vline", label=L"AH", linewidth=1, color = "black", frame = true)
